# PCA & PCA from scratch

In [84]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [85]:
# Load hand-written digits data 
digits = datasets.load_digits()
X = digits.data
y = digits.target

### Before PCA

In [86]:
# Instantiate NB Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NB = GaussianNB()
# Train NB Model
NB.fit(X_train, y_train)

GaussianNB()

In [87]:
# Print Accurucay
y_pred = NB.predict(X_test)

accuracy = accuracy_score(y_pred, y_test)
print(f'Accuracy before PCA: {accuracy*100: .2f} %')

Accuracy before PCA:  84.72 %


### After PCA

In [88]:
# Use PCA to reduce number of features to 10
pca = PCA(n_components=10)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [89]:
# Instantiate second NB model
NB_PCA = GaussianNB()
# Train second model using new features
NB_PCA.fit(X_train_pca, y_train)


GaussianNB()

In [90]:
# Print new accuracy
y_pred_pca = NB_PCA.predict(X_test_pca)

accuracy_pca = accuracy_score(y_pred_pca, y_test)
print(f'Accuracy after PCA: {accuracy_pca*100: .2f} %')


Accuracy after PCA:  91.39 %


### PCA from scratch

In [91]:
import numpy as np
# Subtract the mean to center the data 
# Centering the data ensures that the mean of each feature becomes zero, 
# which is a fundamental step in PCA. It allows PCA to capture the variance in the data.
def center_data(X):
    mean = np.mean(X, axis=0)
    centered_data = X - mean
    return centered_data, mean
# Compute the covariance matrix
# The covariance matrix represents how features in the data are related to each other. 
# PCA aims to find orthogonal directions (principal components) along which the data varies the most, 
# and these directions correspond to the eigenvectors of the covariance matrix.
def covariance_matrix(X):
    cov_matrix = np.cov(X, rowvar=False) # columns represent variables and rows represent observations.
    return cov_matrix
# Calculate Eigenvalues and Eigenvectors of the covariance matrix
# Eigenvalues represent the amount of variance explained by each eigenvector (principal component). 
# Eigenvectors are the directions of maximum variance in the data.
def eigenvalues_eigenvectors(cov_matrix):
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    return eigenvalues, eigenvectors
# Sort the eigenvalues in descending order to get the sorted eigenvectors
# Sorting ensures that the principal components are in descending order of importance (variance explained). 
# The eigenvectors with larger eigenvalues capture more variance.
def sort_eigenvalues_eigenvectors(eigenvalues, eigenvectors):
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    return sorted_eigenvalues, sorted_eigenvectors
# Select the first n eigenvectors
# This step allows you to choose how many principal components you want to retain. 
# A smaller n means more dimensionality reduction.
def first_n_eigenvectors(sorted_eigenvectors, n):
    top_n_eigenvectors = sorted_eigenvectors[:, :n]
    return top_n_eigenvectors
# Transform the data by applying a dot product 
# This step projects the original data into the principal component space, reducing its dimensionality while retaining most of the variance.
def transform_data(X, eigenvectors):
    transformed_data = X.dot(eigenvectors)
    return transformed_data

In [92]:
centered_data, mean = center_data(X_train)

cov_matrix = covariance_matrix(centered_data)

eigenvalues, eigenvectors = eigenvalues_eigenvectors(cov_matrix)

sorted_eigenvalues, sorted_eigenvectors = sort_eigenvalues_eigenvectors(eigenvalues, eigenvectors)

n_components = 10  
top_n_eigenvectors = first_n_eigenvectors(sorted_eigenvectors, n_components)

transformed_X_train = transform_data(centered_data, top_n_eigenvectors)
transformed_X_test = transform_data(X_test - mean, top_n_eigenvectors)

gnb = GaussianNB()
gnb.fit(transformed_X_train, y_train)

y_pred_ = gnb.predict(transformed_X_test)

accuracy_ = accuracy_score(y_test, y_pred_)
print(f'Accuracy after from scratch PCA: {accuracy_*100: .2f} %')

Accuracy after from scratch PCA:  91.39 %


### PCA from scratch - OOP

In [93]:
import numpy as np

class PCA_SC:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
        self.mean = None
    
    def fit(self, X):
        # Calculate the mean of the data
        self.mean = np.mean(X, axis=0)
        
        # Center the data
        X_centered = X - self.mean
        
        # Calculate the covariance matrix
        cov_matrix = np.cov(X_centered, rowvar=False)
        
        # Calculate the eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        
        # Sort eigenvectors by eigenvalues in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[sorted_indices]
        eigenvectors = eigenvectors[:, sorted_indices]
        
        # Select the top n_components eigenvectors
        self.components = eigenvectors[:, :self.n_components]
    
    def transform(self, X):
        # Center the data
        X_centered = X - self.mean
        
        return np.dot(X_centered, self.components)


In [94]:
n_components = 2  # Number of principal components to keep
pca = PCA_SC(n_components)
pca.fit(X_train)

# Transform the training and testing data
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

gnb = GaussianNB()
gnb.fit(X_train_pca, y_train)

y_pred = gnb.predict(X_test_pca)

accuracy = accuracy_score(y_pred, y_test)
print(f'Accuracy after PCA from scratch: {accuracy_*100: .2f} %')

Accuracy after PCA from scratch:  91.39 %
